# 02-03 : Combine Twitter Data

In [1]:
import pandas as pd
import json
import pickle
from pprint import pprint

## Data Load

In [2]:
# Load pickled data
with open('../../data/raw/02_01_tweets.pkl', 'rb') as f:
    tweets = pickle.load(f)

print('Number of tweets: {}'.format(len(tweets)))

Number of tweets: 983


In [3]:
# get the json data from the tweets
tweets_json = [json.loads(tweet.json()) for tweet in tweets]

## Convert to DataFrame

In [4]:
# convert to dataframe
df_tweets = pd.DataFrame(tweets_json)

# show the results
print(df_tweets.shape)
display(df_tweets.head())

(983, 28)


,id,id_str,url,date,user,lang,rawContent,replyCount,retweetCount,likeCount,...,quotedTweet,place,coordinates,inReplyToTweetId,inReplyToUser,source,sourceUrl,sourceLabel,media,_type
0,1684937340169375744,1684937340169375744,https://twitter.com/TopEditorInt/status/168493...,2023-07-28 14:42:21+00:00,"{'id': 22759776, 'id_str': '22759776', 'url': ...",en,"Serious question:\n\nHas anyone, anywhere ever...",0,1,2,...,None,None,None,NaN,None,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.Tweet
1,1684892139375710208,1684892139375710208,https://twitter.com/AkonaMhlana/status/1684892...,2023-07-28 11:42:44+00:00,"{'id': 41370409, 'id_str': '41370409', 'url': ...",en,@Vodacom That’s why I wanted to talk to someon...,1,0,0,...,None,None,None,1.684892e+18,"{'id': 14574763, 'username': 'Vodacom', 'displ...","<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.Tweet
2,1684890636997029888,1684890636997029888,https://twitter.com/AkonaMhlana/status/1684890...,2023-07-28 11:36:46+00:00,"{'id': 41370409, 'id_str': '41370409', 'url': ...",en,@Vodacom what number should I dial to talk to...,1,0,0,...,None,None,None,NaN,"{'id': 14574763, 'username': 'Vodacom', 'displ...","<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.Tweet
3,1684595913744953345,1684595913744953345,https://twitter.com/Moselanku/status/168459591...,2023-07-27 16:05:38+00:00,"{'id': 198678570, 'id_str': '198678570', 'url'...",en,"If you knew how useless your chatbot is , you ...",1,0,0,...,None,None,None,NaN,None,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.Tweet
4,1683916251821756416,1683916251821756416,https://twitter.com/PoliteMashaba07/status/168...,2023-07-25 19:04:54+00:00,"{'id': 1062269709696024576, 'id_str': '1062269...",en,That Vodacom Tobi is useless when doing sim swap.,0,0,0,...,None,None,None,NaN,None,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,"{'photos': [], 'videos': [], 'animated': []}",snscrape.modules.twitter.Tweet


In [7]:
pprint(tweets_json[101])

{'_type': 'snscrape.modules.twitter.Tweet',
 'cashtags': [],
 'conversationId': 1643599377834061826,
 'coordinates': None,
 'date': '2023-04-07 15:08:10+00:00',
 'hashtags': [],
 'id': 1644356395607678977,
 'id_str': '1644356395607678977',
 'inReplyToTweetId': 1643599377834061826,
 'inReplyToUser': {'_type': 'snscrape.modules.twitter.UserRef',
                   'displayname': 'Vodacom',
                   'id': 14574763,
                   'username': 'Vodacom'},
 'lang': 'en',
 'likeCount': 0,
 'links': [],
 'media': {'animated': [], 'photos': [], 'videos': []},
 'mentionedUsers': [{'_type': 'snscrape.modules.twitter.UserRef',
                     'displayname': 'Vodacom',
                     'id': 14574763,
                     'username': 'Vodacom'},
                    {'_type': 'snscrape.modules.twitter.UserRef',
                     'displayname': 'Evan Greenwood',
                     'id': 2867399789,
                     'username': 'CodeOfTheVoid'},
                    {'_t